In [19]:
# nest_asyncio는 Jupyter Notebook에서 asyncio 이벤트 루프를 다시 적용할 수 있게 합니다.
import nest_asyncio

nest_asyncio.apply()


In [9]:
# JSON 페이로드의 예상 구조를 정의하는 Pydantic 모델
from pydantic import BaseModel

class Query(BaseModel):
  prompt: str
  stroke: str
  distance: int
  speed: int
  technique: str    

In [20]:
# security 파일 불러오기
import os
import json

def load_secrets():
    # Jupyter Notebook에서는 __file__ 변수를 사용할 수 없으므로 os.getcwd()를 사용
    script_dir = os.getcwd()
    file_path = os.path.join(script_dir, '../test-security.json')  # 상대 경로 수정
    with open(file_path, 'r') as file:
        return json.load(file)


In [21]:
# llm 객체 생성
from langchain.chat_models import ChatOpenAI

def get_llm_model(name="gpt-4o-mini"):
    # llm 객체 생성
    llm = ChatOpenAI(
        temperature=0.1,  # 창의성 (0.0 ~ 2.0)
        max_tokens=2048,  # 최대 토큰수
        model_name=name,  # 모델명
    )
    return llm

In [22]:
# template을 이용하여 prompt 객체 생성
from langchain.prompts import PromptTemplate

def get_prompt():
  # template 정의
  template = """
    다음은 수영 영법별 수준을 평가하는 기준입니다. md 태그로 제공합니다.
    -----
      # 수영 루틴

      ## 자유형

      ### 거리 (50분 동안 완주 가능 거리)
      - **초급**: 400m 보다 작음
      - **중급**: 800m 보다 작음
      - **상급**: 800m 와 같거나 큼

      ### 속도 (50m 완주)
      - **초급**: 60초 와 같거나 큼
      - **중급**: 45초 보다 크거나 같고, 60초 보다 작음
      - **상급**: 45초 보다 작음

      ### Technique
      - **초급**: 기본 자세
      - **중급**: 양측 호흡, 사이드 턴
      - **상급**: 플립 턴

      ## 배영

      ### 거리 (50분 동안 완주 가능 거리)
      - **초급**: 300m 보다 작음
      - **중급**: 500m 보다 작음
      - **상급**: 500m 와 같거나 큼

      ### 속도 (50m 완주)
      - **초급**: 75초 와 같거나 큼
      - **중급**: 55초 보다 크거나 같고, 75초 보다 작음
      - **상급**: 55초 보다 작음

      ### 기술
      - **초급**: 기본 자세
      - **중급**: 안정적 자세, 배영 턴
      - **상급**: 크로스 오버 턴

      ## 평영

      ### 거리 (50분 동안 완주 가능 거리)
      - **초급**: 300m 보다 작음
      - **중급**: 500m 보다 작음
      - **상급**: 500m 와 같거나 큼

      ### 속도 (50m 완주)
      - **초급**: 80초 와 같거나 큼
      - **중급**: 60초 보다 크거나 같고, 80초 보다 작음
      - **상급**: 60초 보다 작음

      ### 기술
      - **초급**: 글라이딩
      - **중급**: 돌핀 킥
      - **상급**: 물 속 스타트

      ## 접영

      ### 거리 (50분 동안 완주 가능 거리)
      - **초급**: 100m 보다 작음
      - **중급**: 300m 보다 작음
      - **상급**: 300m 와 같거나 큼

      ### 속도 (50m 완주)
      - **초급**: 70초 와 같거나 큼
      - **중급**: 50초 보다 크거나 같고, 70초 보다 작음
      - **상급**: 50초 보다 작음

      ### 기술
      - **초급**: 한팔 접영
      - **중급**: 양팔 접영
      - **상급**: 무호흡 양팔 접영 (25m)

      다음은 각 기준별 중요도를 퍼센테이지로 나타낸 것입니다.
      # 중요도
      - 거리: 40,
      - 속도: 35,
      - 기술: 25
      -----
      출력형식: JSON
      json 속성: stroke, level, description
      -----
      instrcution: 위 기준을 참고하여 질의에 답변해 주세요. 만약 명확하지 않다면 판정불가 판정을 내려주세요.
      ------
      질문: {stroke}의 경우 50분 동안 {distance}미터를 수영할 수 있고 속도는 {speed}초입니다. 최근 {technique} 기술을 배웠습니다.
    """
  prompt = PromptTemplate.from_template(template=template)
  return prompt

In [23]:
# llm chain 객체 생성
from langchain.chains import LLMChain

def get_llm_chain():
  return LLMChain(prompt=get_prompt(), llm=get_llm_model())

In [24]:
from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.post("/prompt/")
async def create_item(item: Query):
  secrets = load_secrets()
  os.environ['OPENAI_API_KEY'] = secrets['OPENAI_API_KEY']

  # 연결된 체인(Chain) 객체 생성
  llm_chain = get_llm_chain()

  # llm_chain에 맞게 inputs 전달
  response = llm_chain.invoke({
      "prompt": item.prompt,
      "stroke": item.stroke,
      "distance": item.distance,
      "speed": item.speed,
      "technique": item.technique
  })

  return {"completion": response}

if __name__ == "__main__":
  uvicorn.run(app, host="localhost", port=8000)


INFO:     Started server process [16320]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
c:\DevelopTools\Python\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\DevelopTools\Python\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


INFO:     127.0.0.1:52568 - "POST /prompt/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16320]
